In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
tf.reset_default_graph()

In [64]:
window_size = 24
num_sensor_data = 113
number_classes = 18
num_units = [window_size, window_size, window_size]

In [65]:
win_size = tf.constant(24, dtype=tf.float32)

In [66]:
# PLACEHOLDERS
X = tf.placeholder(tf.float32, shape=[None,window_size,num_sensor_data])
y = tf.placeholder(tf.float32, shape=[None, number_classes])
hold_prob = tf.placeholder(tf.float32);

In [67]:
def LSTM_with_Dropout(num_units, hold):
    _cell = tf.nn.rnn_cell.LSTMCell(num_units)
    
    return _cell

In [68]:
fw_cell = [LSTM_with_Dropout(n, hold_prob) for n in num_units]
bw_cell = [LSTM_with_Dropout(n, hold_prob) for n in num_units]

In [69]:
bidi_output, _ , _ = tf.contrib.rnn.stack_bidirectional_rnn(fw_cell, bw_cell, tf.unstack(X,num_sensor_data,2), dtype=tf.float32)

In [70]:
#fusion_output = tf.convert_to_tensor([tf.reduce_sum(t) / win_size for t in bidi_output])
fusion_output_1 = tf.reshape(bidi_output, shape=[-1, num_sensor_data*window_size*2])
print(fusion_output_1)

Tensor("Reshape:0", shape=(?, 5280), dtype=float32)


In [71]:
dense_output1 = tf.contrib.layers.fully_connected(fusion_output_1, 2640, activation_fn=tf.nn.relu)

In [72]:
dense_output2 = tf.contrib.layers.fully_connected(dense_output1, 1320, activation_fn=tf.nn.relu)

In [73]:
dense_output3 = tf.contrib.layers.fully_connected(dense_output2, 18, activation_fn=tf.nn.softmax)

In [78]:
cross_entropy = tf.reduce_mean(tf.squared_difference(y, dense_output3))

In [86]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [87]:
init = tf.global_variables_initializer()

In [ ]:
# model saver

In [89]:
steps = 10000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x , batch_y = train_helper.get_train_batch()
        
        sess.run(train,feed_dict={X:batch_x,y:batch_y,hold_prob:0.9})
        
        if i%100 == 0:
            print('Training is on step {}'.format(i))
    
    print('Training finished')
    print('Evaluating on training')
    test_steps = 500
    count = 0.0
    den = 0.0
    for m in range(test_steps):
        den = den + 1.0
        batch_x , batch_y = new_train_helper.get_train_batch() 
        output = dense_output3.eval(feed_dict={X:batch_x,y:batch_y,hold_prob:1.0})
        match = np.argmax(output) == np.argmax(batch_y)
        match = match.astype(float)
        count = count + match
        
        if m%100 == 0:
            print('Testing is on step {}'.format(m))
            print('Partial Accuracy')
            print(count / den)
    
    print('Testing finished')

Training is on step 0
Training is on step 100
Training is on step 200
Training is on step 300
Training is on step 400
Training is on step 500
Training is on step 600
Training is on step 700
Training is on step 800
Training is on step 900
Training is on step 1000
Training is on step 1100
Training is on step 1200
Training is on step 1300
Training is on step 1400
Training is on step 1500
Training is on step 1600
Training is on step 1700
Training is on step 1800
Training is on step 1900
Training is on step 2000
Training is on step 2100
Training is on step 2200
Training is on step 2300
Training is on step 2400
Training is on step 2500
Training is on step 2600
Training is on step 2700
Training is on step 2800
Training is on step 2900
Training is on step 3000
Training is on step 3100
Training is on step 3200
Training is on step 3300
Training is on step 3400
Training is on step 3500
Training is on step 3600
Training is on step 3700
Training is on step 3800
Training is on step 3900
Training is 

In [90]:
print(count/den)

0.342
